In [1]:

from ibmdbpy import IdaDataBase, IdaDataFrame

# @hidden_cell
# This connection object is used to access your data and contains your credentials.
# You might want to remove those credentials before you share your notebook.
idadb = IdaDataBase(dsn='DASHDB;Database=BLUDB;Hostname=dashdb-entry-yp-dal10-01.services.dal.bluemix.net;Port=50000;PROTOCOL=TCPIP;UID=dash5014;PWD=85y_E_UgviMC', verbose=True)

ida_df_1 = IdaDataFrame(idadb, 'GOSALESDW.SLS_SALES_FACT')
ida_df_1.head()

# You can close the database connection with the following code. Please keep the comment line with the @hidden_cell tag,
# because the close function displays parts of the credentials.
# @hidden_cell
# idadb_22ff48a8fcc54518aa9b4b78f9a913e4.close()
# To learn more about the ibmdby package, please read the documentation: http://pythonhosted.org/ibmdbpy/


> SELECT distinct TABSCHEMA, TABNAME, OWNER, TYPE from SYSCAT.TABLES WHERE (OWNERTYPE = 'U') ORDER BY "TABSCHEMA","TABNAME"
> SELECT COLNAME, TYPENAME FROM SYSCAT.COLUMNS WHERE TABNAME='SLS_SALES_FACT' AND TABSCHEMA='GOSALESDW' ORDER BY COLNO
> SELECT * FROM GOSALESDW.SLS_SALES_FACT ORDER BY "ORDER_DAY_KEY" ASC FETCH FIRST 5 ROWS ONLY


,ORDER_DAY_KEY,ORGANIZATION_KEY,EMPLOYEE_KEY,RETAILER_KEY,RETAILER_SITE_KEY,PRODUCT_KEY,PROMOTION_KEY,ORDER_METHOD_KEY,SALES_ORDER_KEY,SHIP_DAY_KEY,CLOSE_DAY_KEY,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE,GROSS_MARGIN,SALE_TOTAL,GROSS_PROFIT
0,20100112,11101,4001,6737,5185,30126,5501,602,194778,20100119,20100119,587,34.90,76.86,71.48,0.5118,41958.76,21472.46
1,20100112,11101,4001,6986,5465,30103,5501,606,194770,20100119,20100119,1776,1.86,6.00,5.70,0.6737,10123.20,6819.84
2,20100112,11101,4001,6986,5465,30100,5501,606,194768,20100119,20100119,129,56.23,96.44,89.69,0.3731,11570.01,4316.34
3,20100112,11101,4001,6737,5185,30132,5501,602,194777,20100119,20100119,576,2.67,10.71,10.28,0.7403,5921.28,4383.36
4,20100112,11101,4001,6737,5185,30128,5501,602,194779,20100119,20100119,214,91.80,180.63,167.99,0.4535,35949.86,16304.66


In [4]:
retailSummary = idadb.ida_query("SELECT CLOSE_DAY_KEY, EMPLOYEE_KEY, GROSS_MARGIN, GROSS_PROFIT, ORDER_DAY_KEY, " +
                                       "ORDER_METHOD_KEY, ORGANIZATION_KEY, PRODUCT_KEY, PROMOTION_KEY, " +
                                       "QUANTITY, RETAILER_KEY RETAILER_SITE_KEY, SALES_ORDER_KEY, SALE_TOTAL, " +
                                       "SHIP_DAY_KEY, UNIT_COST, UNIT_PRICE, UNIT_SALE_PRICE " +
                                "FROM GOSALESDW.SLS_SALES_FACT " +
                                "WHERE QUANTITY > 9000 " +
                                "ORDER BY GROSS_PROFIT DESC")
numberofSales = idadb.ida_scalar_query("SELECT count(*) FROM GOSALESDW.EMP_EXPENSE_FACT")

> SELECT CLOSE_DAY_KEY, EMPLOYEE_KEY, GROSS_MARGIN, GROSS_PROFIT, ORDER_DAY_KEY, ORDER_METHOD_KEY, ORGANIZATION_KEY, PRODUCT_KEY, PROMOTION_KEY, QUANTITY, RETAILER_KEY RETAILER_SITE_KEY, SALES_ORDER_KEY, SALE_TOTAL, SHIP_DAY_KEY, UNIT_COST, UNIT_PRICE, UNIT_SALE_PRICE FROM GOSALESDW.SLS_SALES_FACT WHERE QUANTITY > 9000 ORDER BY GROSS_PROFIT DESC
> SELECT count(*) FROM GOSALESDW.EMP_EXPENSE_FACT


In [6]:
print("Number of total sales: " + str(numberofSales))
print("Retail Summary: ")
retailSummary.tail()

Number of total sales: 127984
Retail Summary: 


,close_day_key,employee_key,gross_margin,gross_profit,order_day_key,order_method_key,organization_key,product_key,promotion_key,quantity,retailer_site_key,sales_order_key,sale_total,ship_day_key,unit_cost,unit_price,unit_sale_price
66,20120310,4051,0.4658,16631.76,20120306,605,11169,30001,5542,9039,7297,162452,35704.05,20120310,2.11,6.59,3.95
67,20120213,4124,0.4658,16629.92,20120209,605,11169,30001,5542,9038,6873,162103,35700.10,20120213,2.11,6.59,3.95
68,20130216,4086,0.4842,16598.64,20130212,605,11109,30050,5501,9021,6868,192430,34279.80,20130216,1.96,4.00,3.80
69,20130423,4086,0.4842,16567.36,20130418,605,11109,30050,5501,9004,6868,193127,34215.20,20130423,1.96,4.00,3.80
70,20130213,4048,0.4506,16082.30,20130115,605,11168,30001,5606,9035,7148,171887,35688.25,20130118,2.17,6.59,3.95


In [10]:
%time
agg1 = idadb.ida_query("SELECT EXTRACT(YEAR FROM CAST(CAST (order_day_key AS varchar(100)) AS timestamp)) AS year, " +
                            "SUM (sale_total) AS total_sales, " + 
                            "RANK () OVER (ORDER BY SUM (sale_total) DESC) AS ranked_sales " + 
                        "FROM gosalesdw.sls_sales_fact " + 
                        "GROUP BY EXTRACT(YEAR FROM CAST(CAST (order_day_key AS varchar(100)) AS timestamp)) ")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 15 µs
> SELECT EXTRACT(YEAR FROM CAST(CAST (order_day_key AS varchar(100)) AS timestamp)) AS year, SUM (sale_total) AS total_sales, RANK () OVER (ORDER BY SUM (sale_total) DESC) AS ranked_sales FROM gosalesdw.sls_sales_fact GROUP BY EXTRACT(YEAR FROM CAST(CAST (order_day_key AS varchar(100)) AS timestamp)) 


In [11]:
%time
agg1

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 15.3 µs


,year,total_sales,ranked_sales
0,2012,1.495891e+09,1
1,2011,1.159196e+09,2
2,2013,1.117336e+09,3
3,2010,9.143528e+08,4


In [4]:
ida_df_1.head(10000).cov()

> SELECT * FROM GOSALESDW.SLS_SALES_FACT ORDER BY "ORDER_DAY_KEY" ASC FETCH FIRST 10000 ROWS ONLY


,ORDER_DAY_KEY,ORGANIZATION_KEY,EMPLOYEE_KEY,RETAILER_KEY,RETAILER_SITE_KEY,PRODUCT_KEY,PROMOTION_KEY,ORDER_METHOD_KEY,SALES_ORDER_KEY,SHIP_DAY_KEY,CLOSE_DAY_KEY,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE,GROSS_MARGIN,SALE_TOTAL,GROSS_PROFIT
ORDER_DAY_KEY,1978.830249,-1.870265e+02,-8.022758e+02,-3.180752e+02,-3.489821e+02,-5.217493e+02,0.867695,-1.465005,-8.597989e+05,1973.795516,2.085177e+03,1.572468e+03,1.501619e+02,3.074550e+02,2.557866e+02,0.209498,6.136397e+04,2.409049e+04
ORGANIZATION_KEY,-187.026496,7.218506e+02,1.340842e+03,3.717800e+02,2.005514e+02,1.251007e+03,-0.586254,3.602064,1.727406e+06,-183.386378,-3.568631e+02,-2.792053e+03,-4.027113e+02,-7.969222e+02,-6.847893e+02,-0.371383,-1.114871e+05,-4.384761e+04
EMPLOYEE_KEY,-802.275832,1.340842e+03,1.085394e+04,2.707024e+03,3.333284e+03,5.569475e+03,-2.497637,14.615045,1.089272e+07,-790.234419,-1.585465e+03,-1.893870e+04,-1.818242e+03,-3.629207e+03,-3.103871e+03,-1.700536,-7.828527e+05,-3.106063e+05
RETAILER_KEY,-318.075188,3.717800e+02,2.707024e+03,4.125491e+04,5.137878e+04,7.273065e+02,0.136413,129.857977,2.326203e+06,-537.291386,-1.335201e+03,2.790036e+03,-1.389371e+03,-2.794132e+03,-2.548475e+03,-0.909835,-2.552664e+04,-2.862293e+04
RETAILER_SITE_KEY,-348.982148,2.005514e+02,3.333284e+03,5.137878e+04,6.805432e+04,5.963677e+02,0.391159,172.787414,2.769631e+06,-605.639580,-1.432400e+03,3.646351e+03,-1.627593e+03,-3.260426e+03,-2.982535e+03,-1.062770,8.236556e+02,-2.309400e+04
PRODUCT_KEY,-521.749296,1.251007e+03,5.569475e+03,7.273065e+02,5.963677e+02,6.133504e+03,-3.505856,11.556414,7.831676e+06,-518.900972,-1.171138e+03,-1.355100e+04,-6.667649e+02,-1.114115e+03,-7.732211e+02,-0.153235,-6.232042e+05,-2.213616e+05
PROMOTION_KEY,0.867695,-5.862540e-01,-2.497637e+00,1.364130e-01,3.911593e-01,-3.505856e+00,0.070552,-0.006465,-3.270205e+03,0.843885,1.250723e+00,1.007163e+01,-8.225851e-01,-1.507510e+00,-1.820477e+00,-0.004383,3.430200e+02,4.403873e+01
ORDER_METHOD_KEY,-1.465005,3.602064e+00,1.461505e+01,1.298580e+02,1.727874e+02,1.155641e+01,-0.006465,2.003830,2.480640e+04,-2.375222,-9.659545e+00,-4.711371e+00,-1.062326e+01,-2.102679e+01,-1.895113e+01,-0.008126,-1.090954e+03,-5.589476e+02
SALES_ORDER_KEY,-859798.880199,1.727406e+06,1.089272e+07,2.326203e+06,2.769631e+06,7.831676e+06,-3270.204704,24806.403652,1.887538e+10,-842971.128897,-2.000241e+06,-2.566140e+07,-2.531166e+06,-5.023362e+06,-4.304225e+06,-2332.158310,-1.040653e+09,-4.131200e+08
SHIP_DAY_KEY,1973.795516,-1.833864e+02,-7.902344e+02,-5.372914e+02,-6.056396e+02,-5.189010e+02,0.843885,-2.375222,-8.429711e+05,2144.256482,2.255682e+03,1.503121e+03,1.333407e+02,2.723949e+02,2.229516e+02,0.187574,5.727400e+04,2.228521e+04


In [18]:
#from ibmdbpy.sampledata.iris import iris
from sklearn.datasets import load_iris
import pandas as pd

In [8]:
iris1 = load_iris()

In [13]:
type(iris1.data)

numpy.ndarray

In [23]:
#iris_tab = idadb.as_idadataframe(pd.DataFrame(iris["data"], columns=iris1["feature_names"]), "IRIS", clear_existing = True)
iris_tab = idadb.as_idadataframe(pd.DataFrame(iris1["data"], columns=["SEPAL_LENGTH","SEPAL_WIDTH","PETAL_LENGTH","PETAL_WIDTH"]), "IRIS", clear_existing = True)

> DROP TABLE IRIS
<< AUTOCOMMIT >>
> CREATE TABLE "IRIS" ("SEPAL_LENGTH" DOUBLE,"SEPAL_WIDTH" DOUBLE,"PETAL_LENGTH" DOUBLE,"PETAL_WIDTH" DOUBLE)
> SELECT distinct TABSCHEMA, TABNAME, OWNER, TYPE from SYSCAT.TABLES WHERE (OWNERTYPE = 'U') ORDER BY "TABSCHEMA","TABNAME"
Uploading 150 rows (maxnrow was set to 2000)
<< AUTOCOMMIT >>
Execution time: 0.319364070892334 seconds.


In [24]:
iris_tab.head()

> SELECT COLNAME, TYPENAME FROM SYSCAT.COLUMNS WHERE TABNAME='IRIS' AND TABSCHEMA='DASH5014' ORDER BY COLNO
> SELECT * FROM DASH5014.IRIS ORDER BY "SEPAL_LENGTH" ASC FETCH FIRST 5 ROWS ONLY


,SEPAL_LENGTH,SEPAL_WIDTH,PETAL_LENGTH,PETAL_WIDTH
0,4.3,3.0,1.1,0.1
1,4.4,3.0,1.3,0.2
2,4.4,2.9,1.4,0.2
3,4.4,3.2,1.3,0.2
4,4.5,2.3,1.3,0.3


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris1.data, iris1.target, random_state = 3)

In [27]:
iris_train = idadb.as_idadataframe(pd.DataFrame(X_train, columns=["SEPAL_LENGTH","SEPAL_WIDTH","PETAL_LENGTH","PETAL_WIDTH"]), "IRIS", clear_existing = True)

> DROP TABLE IRIS
<< AUTOCOMMIT >>
> CREATE TABLE "IRIS" ("SEPAL_LENGTH" DOUBLE,"SEPAL_WIDTH" DOUBLE,"PETAL_LENGTH" DOUBLE,"PETAL_WIDTH" DOUBLE)
> SELECT distinct TABSCHEMA, TABNAME, OWNER, TYPE from SYSCAT.TABLES WHERE (OWNERTYPE = 'U') ORDER BY "TABSCHEMA","TABNAME"
Uploading 112 rows (maxnrow was set to 2000)
<< AUTOCOMMIT >>
Execution time: 0.3726348876953125 seconds.


In [29]:
ida_df_1["PROMOTION_KEY"].count()

> SELECT COUNT("PROMOTION_KEY") FROM (SELECT "PROMOTION_KEY" FROM GOSALESDW.SLS_SALES_FACT)
Execution time: 0.06115102767944336 seconds.


446023

In [30]:
from ibmdbpy.sampledata.iris import iris

In [31]:
iris_tab2 = idadb.as_idadataframe(iris, "IRIS2", clear_existing = True)

> CREATE TABLE "IRIS2" ("sepal_length" DOUBLE,"sepal_width" DOUBLE,"petal_length" DOUBLE,"petal_width" DOUBLE,"species" VARCHAR(255))
Uploading 150 rows (maxnrow was set to 1600)
<< AUTOCOMMIT >>
Execution time: 0.3576629161834717 seconds.


In [32]:
y_train

array([2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 2, 1, 0, 0, 2, 1, 1, 0, 2, 0, 2, 1, 0,
       0, 2, 1, 0, 0, 1, 2, 2, 0, 2, 1, 0, 0, 2, 2, 2, 1, 1, 1, 0, 0, 2, 2,
       1, 2, 1, 2, 0, 2, 0, 1, 1, 2, 2, 0, 1, 0, 1, 1, 1, 0, 2, 0, 2, 1, 2,
       1, 2, 1, 0, 2, 1, 2, 1, 0, 1, 2, 0, 1, 0, 0, 0, 1, 2, 0, 0, 2, 0, 1,
       2, 1, 2, 2, 1, 1, 2, 1, 0, 1, 1, 0, 1, 2, 2, 2, 0, 0, 2, 2])

In [34]:
iris_tab2["species"].head()

> CREATE VIEW "TEMP_VIEW_IRIS2_23836_1527711841" AS (SELECT "species" FROM DASH5014.IRIS2)
<< AUTOCOMMIT >>
> SELECT COLNAME, TYPENAME FROM SYSCAT.COLUMNS WHERE TABNAME='TEMP_VIEW_IRIS2_23836_1527711841' ORDER BY COLNO
> DROP VIEW "TEMP_VIEW_IRIS2_23836_1527711841"
<< AUTOCOMMIT >>
> SELECT * FROM (SELECT "species" FROM DASH5014.IRIS2) ORDER BY "species" ASC FETCH FIRST 5 ROWS ONLY


0    setosa
1    setosa
2    setosa
3    setosa
4    setosa
Name: species, dtype: object